<a href="https://colab.research.google.com/github/jdwatson08/Insurance_Purchase_Model/blob/main/Insurance_Purchase_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive/Kaggle/all_state

/content/gdrive/My Drive/Kaggle/all_state


In [4]:
!unzip \*.zip && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [5]:
!ls

sampleSubmission.csv  test_v2.csv  train.csv


In [6]:
def wrangle(filename):
  df =pd.read_csv(filename)


  #Drop high cardinality columns
  cols_to_drop=[]
  threshold=10000
  for col in df:
    if df[col].dtype == 'object':
      if df[col].nunique() > threshold:
        cols_to_drop.append(col)

  df.drop(columns=cols_to_drop, inplace=True)

  #Remove columns with majority null values
  threshold = 1000
  null_cols =[]
  for col in df:
    if df[col].isnull().sum() > threshold:
      null_cols.append(col)

  df.drop(columns= null_cols, inplace=True)

  #Remove columns with only 1 value
  one_val =[]
  for col in df:
    if df[col].nunique()==1:
      one_val.append(col)

  df.drop(columns=one_val, inplace=True)

  return df

In [7]:
train_data = wrangle('train.csv')
test_data = wrangle('test_v2.csv')

In [8]:
train_data

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,age_youngest,married_couple,A,B,C,D,E,F,G,cost
0,10000000,1,0,0,08:35,IN,10001,2,0,2,...,42,1,1,0,2,2,1,2,2,633
1,10000000,2,0,0,08:38,IN,10001,2,0,2,...,42,1,1,0,2,2,1,2,1,630
2,10000000,3,0,0,08:38,IN,10001,2,0,2,...,42,1,1,0,2,2,1,2,1,630
3,10000000,4,0,0,08:39,IN,10001,2,0,2,...,42,1,1,0,2,2,1,2,1,630
4,10000000,5,0,0,11:55,IN,10001,2,0,2,...,42,1,1,0,2,2,1,2,1,630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665244,10152724,2,0,3,13:43,KY,10204,1,1,1,...,20,0,1,0,2,3,0,2,2,677
665245,10152724,3,0,3,13:43,KY,10204,1,1,1,...,20,0,1,0,2,3,0,2,2,677
665246,10152724,4,0,3,13:44,KY,10204,1,1,1,...,20,0,1,0,2,3,0,2,2,677
665247,10152724,5,0,3,13:46,KY,10204,1,1,1,...,20,0,1,0,2,3,0,2,2,685


In [9]:
train_data['time'].dtype

dtype('O')

In [10]:
train_data['cost'].describe().to_frame

<bound method Series.to_frame of count    665249.000000
mean        635.785008
std          45.993758
min         260.000000
25%         605.000000
50%         635.000000
75%         665.000000
max         922.000000
Name: cost, dtype: float64>

In [11]:
train_data.describe()

,customer_ID,shopping_pt,record_type,day,location,group_size,homeowner,car_age,age_oldest,age_youngest,married_couple,A,B,C,D,E,F,G,cost
count,6.652490e+05,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000,665249.000000
mean,1.007655e+07,4.219966,0.145824,1.969429,12271.543022,1.234784,0.536229,8.139437,44.992403,42.577588,0.209782,0.927546,0.454236,2.275265,2.454299,0.445193,1.162768,2.275120,635.785008
std,4.404978e+04,2.394369,0.352930,1.453470,1564.789415,0.461036,0.498686,5.764598,17.403440,17.460432,0.407153,0.595221,0.497902,0.990798,0.753953,0.496987,0.943594,0.909567,45.993758
min,1.000000e+07,1.000000,0.000000,0.000000,10001.000000,1.000000,0.000000,0.000000,18.000000,16.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,260.000000
25%,1.003852e+07,2.000000,0.000000,1.000000,10936.000000,1.000000,0.000000,3.000000,28.000000,26.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,2.000000,605.000000
50%,1.007640e+07,4.000000,0.000000,2.000000,12027.000000,1.000000,1.000000,7.000000,44.000000,40.000000,0.000000,1.000000,0.000000,2.000000,3.000000,0.000000,1.000000,2.000000,635.000000
75%,1.011470e+07,6.000000,0.000000,3.000000,13426.000000,1.000000,1.000000,12.000000,60.000000,57.000000,0.000000,1.000000,1.000000,3.000000,3.000000,1.000000,2.000000,3.000000,665.000000
max,1.015272e+07,13.000000,1.000000,6.000000,16580.000000,4.000000,1.000000,85.000000,75.000000,75.000000,1.000000,2.000000,1.000000,4.000000,3.000000,1.000000,3.000000,4.000000,922.000000


In [12]:
bew_table = {}
for feat in train_data:
  print((feat: train_data[feat].nunique()))

SyntaxError: invalid syntax (<ipython-input-12-1107e6d007b3>, line 3)

In [ ]:
new_features = ['A','B','C','D','E','F','G']
cost_table = train_data[new_features]
#plt.hist(data = cost_table, x = new_features, bins = 20)
plt.bar(data = cost_table, x = new_features, height = 1000000)
plt.show()

In [ ]:
cost_table['A'].count()

In [ ]:
665250